In [28]:
import torch
import numpy as np
import pandas as pd
from transformers import LlamaTokenizer, LlamaForSequenceClassification, LlamaModel, LlamaForCausalLM, AutoConfig, AutoModel
from accelerate import infer_auto_device_map, init_empty_weights

from src.utils import *
from datasets import Dataset
from src.models.baseline import Baseline
from torch.nn.functional import one_hot
from torch.utils.data import DataLoader

In [2]:
RANDOM_SEED=42

url = "https://drive.google.com/uc?export=download&id=1wVNU2XvvhqjaGXZM-JLJwOt97gt4g9j2"
dataset_name = "MELD_train_efr.json"

df_manager = DataframeManager(url, dataset_name)

df = df_manager.produce_df()
df

Current work directory: c:\Users\marco\OneDrive\Immagini\Documenti\GitHub\ediref


,episode,emotions,utterances,triggers,emotions_labels
0,utterance_0,"[neutral, neutral, neutral, neutral, surprise]",[also I was the point person on my company's t...,"[0, 0, 0, 1, 0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0...."
1,utterance_1,"[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0, 0, 0, 0, 0, 1, 0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0...."
2,utterance_2,"[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0...."
3,utterance_3,"[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0...."
4,utterance_4,"[surprise, sadness, surprise, fear]",[But then who? The waitress I went out with la...,"[0, 0, 1, 0]","[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 1...."
...,...,...,...,...,...
3995,utterance_3995,"[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0...."
3996,utterance_3996,"[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0...."
3997,utterance_3997,"[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0...."
3998,utterance_3998,"[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0...."


In [3]:
train_df, val_df, test_df = df_manager.split_df(RANDOM_SEED)
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(3200, 5)
(400, 5)
(400, 5)


In [15]:
raw_df = pd.read_json(url)
raw_df = pd.DataFrame(raw_df)
dataset = Dataset.from_pandas(raw_df)

In [17]:
dataset[0]

{'episode': 'utterance_0',
 'speakers': ['Chandler',
  'The Interviewer',
  'Chandler',
  'The Interviewer',
  'Chandler'],
 'emotions': ['neutral', 'neutral', 'neutral', 'neutral', 'surprise'],
 'utterances': ["also I was the point person on my company's transition from the KL-5 to GR-6 system.",
  "You must've had your hands full.",
  'That I did. That I did.',
  "So let's talk a little bit about your duties.",
  'My duties?  All right.'],
 'triggers': [0.0, 0.0, 0.0, 1.0, 0.0]}

In [4]:
# train_df.iloc[1]["utterances"]
train_df

,episode,emotions,utterances,triggers,emotions_labels
555,utterance_555,"[sadness, anger]","[Look, I feel really bad about how I freaked y...","[0, 0]","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0...."
3491,utterance_3491,"[surprise, fear, surprise, sadness]","[You-you you had sex with Ursula?!, Uh, a litt...","[0, 0, 1, 0]","[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0...."
527,utterance_527,"[fear, neutral, anger, sadness]","[Oh my God! Oh my God! I'm so sorry!, Aw forge...","[0, 0, 1, 0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0...."
3925,utterance_3925,"[neutral, neutral, neutral, disgust]","[I can blow dry it., I can put gel on it., It ...","[0, 0, 1, 0]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0...."
2989,utterance_2989,"[joy, joy, neutral]",[You're gonna love me so much. I got Sting tic...,"[0, 1, 0]","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0...."
...,...,...,...,...,...
3939,utterance_3939,"[joy, neutral, neutral, surprise, sadness, neu...","[Well, I feel like a snack!, Do you want some ...","[0, 0, 0, 0, 1, 0]","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0...."
2523,utterance_2523,"[neutral, neutral, neutral, sadness, neutral, ...","[So, um, what do you do for a living?, Well, u...","[0, 0, 0, 0, 0, 0, 1]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0...."
1683,utterance_1683,"[joy, neutral, anger, surprise, sadness, surpr...","[Chandler, I found the perfect ring., Oh, that...","[0, 0, 0, 0, 0, 0, 1]","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0...."
3422,utterance_3422,"[anger, neutral, neutral, surprise, anger, dis...","[Hey! Hold on a minute, hold on a second. Do y...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0...."


In [79]:
class EFRDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        print(self.df.iloc[idx]["utterances"])
        return self.df.iloc[idx]["utterances"], self.df.iloc[idx]["triggers"], self.df.iloc[idx]["emotions_labels"]

In [77]:
train_data = EFRDataset(train_df)
train_dataloader = DataLoader(train_data, batch_size=1, shuffle=False)

In [75]:
train_df.iloc[0]["utterances"]

['Look, I feel really bad about how I freaked you out before, so I called the father and asked him to meet you here so you can tell him.',
 'Go!']

In [78]:
for batch_idx, data in enumerate(train_dataloader):
    print(data)
    break

KeyError: 1

In [2]:
model_path = 'openlm-research/open_llama_3b_v2'
tokenizer = LlamaTokenizer.from_pretrained(model_path)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
# config = AutoConfig.from_pretrained(model_path)
# with init_empty_weights():
#     model = AutoModel.from_config(config)
# device_map = infer_auto_device_map(model)

model = LlamaModel.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto', offload_folder="offload_base", offload_state_dict=True
)

In [9]:
prompt = 'Q: What is the largest animal?\nA:'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

output = model.generate(input_ids)
# output = class_model(input_ids)

# generation_output = causal_model.generate(
#     input_ids=input_ids, max_new_tokens=32
# )
# print(tokenizer.decode(generation_output[0]))

TypeError: The current model class (LlamaModel) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'LlamaForCausalLM'}

https://huggingface.co/blog/accelerate-large-models